In [1]:
# =====================================================
# 🏠 House Prices - Modelos Lineares
# =====================================================
import pandas as pd
from pathlib import Path
# Scikit-learn - Modelos lineares
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from setup_notebook import setup_path
setup_path()
from src.model_utils import *
import joblib
import matplotlib.pyplot as plt
import os

from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

from scipy.stats import loguniform

from sklearn.model_selection import cross_validate
import numpy as np
import time


import warnings

warnings.filterwarnings("ignore")
from setup_notebook import setup_path
setup_path()
from src.model_utils import *
import joblib
import matplotlib.pyplot as plt
import os

In [2]:
BASE = Path.cwd().parent   
# =====================================================
# ⚙️ 0. carregamento dos preprocessador 
# =====================================================
temp = joblib.load(BASE /'src'/'preprocess_house_prices_v1.joblib')
preprocessador=temp['preprocessador']
colnull_train = temp['colnull_train']

# =====================================================
# 📁 1. Leitura dos dados & Separação das bases
# =====================================================
DATA_DIR = BASE / "data" / "raw"
X_train = pd.read_csv(DATA_DIR / "X_train_raw.csv")
X_test  = pd.read_csv(DATA_DIR / "X_test_raw.csv")
y_train = pd.read_csv(DATA_DIR / "y_train_raw.csv")
y_test  = pd.read_csv(DATA_DIR / "y_test_raw.csv")


#MODELOS
model_xg0    = XGBRegressor( objective='reg:squarederror',random_state=42,n_jobs=-1 ) 
XGB0         = pipe_models(model_xg0 , preprocessador)

model_xg1    = XGBRegressor( objective='reg:squarederror', n_estimators=1000,
                         subsample= 0.8,max_depth= 6,learning_rate= 0.1,
                         colsample_bytree= 0.8,random_state=42,n_jobs=-1 ) 
XGB1         = pipe_models(model_xg1 , preprocessador)

model_xg2    = XGBRegressor( objective='reg:squarederror', n_estimators=700,
                         subsample= 0.6,reg_lambda= 0.5,reg_alpha= 1.0,
                         max_depth= 3,learning_rate= 0.073,
                         colsample_bytree= 0.7,
                         n_jobs=-1 ) 
XGB2         = pipe_models(model_xg2 , preprocessador)

#SALVAR
DATA_MODELS= BASE /"models"

joblib.dump(XGB0,DATA_MODELS / 'modelo_XGB0_final_v1.joblib')
joblib.dump(XGB1,DATA_MODELS / 'modelo_XGB1_final_v1.joblib')
joblib.dump(XGB2,DATA_MODELS / 'modelo_XGB2_final_v1.joblib')

['/home/akel/PycharmProjects/Kaggle/HousePrices/models/modelo_XGB2_final_v1.joblib']